# Import Libraries

In [ ]:
# importing the libraries
import pandas as pd
import numpy as np
import os
import cv2
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
%matplotlib inline

from IPython.display import clear_output

# for reading and displaying images
from skimage import io
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import Linear, ReLU, Tanh, Hardtanh, LeakyReLU, ELU, Sigmoid, CrossEntropyLoss, Sequential, Conv2d, ConvTranspose2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torch.nn.utils import spectral_norm
import torch.nn.functional as f
import torchvision.transforms as transforms
from torch.optim.swa_utils import AveragedModel, SWALR
from scipy.stats import mode
from scipy import stats, optimize

In [ ]:
data = os.uname()
username = os.getlogin()
global device
if data[1] == 'lambda-dual':
    device = 'cuda:1'
    print('The model will be trained on {}'.format(device))
    data_dir = '/home/'+username+'/engrprojects/WilsonGroup/TuCo/data/20210920 modify SVD_mods_Arya/620nm/'
    save_dir = '/home/'+username+'/engrprojects/WilsonGroup/MRA_grant/work_in_progress_arya/spectral_autoencoder_data/'
    print('Data will be loaded from "{}"'.format(data_dir))
    print('Data will be saved to "{}"'.format(save_dir))
elif data[1] == 'linuxg3' or data[1] == 'linuxg4':
    device = 'cuda:0'
    data_dir = '/top/projects/WilsonGroup/TuCo/data/20210920 modify SVD_mods_Arya/620nm/'
    save_dir = '/top/projects/WilsonGroup/MRA_grant/work_in_progress_arya/spectral_autoencoder_data/'
    print('The model will be trained on {}'.format(device))
    print('Data will be saved to "{}"'.format(save_dir))
    

In [ ]:
device='cuda:0'

# Common visualization functions

In [ ]:
def orthogonalColorPalette( n ):
    # generates a list of n colors, with even hue angle spacing between
    h = np.linspace(0,1,n+1)[0:-1]  # evenly spaced hue angles
    s = np.ones_like(h)             # saturation = 1 
    v = s                           # value = 1
    hsv = np.stack((h,s,v)).T       # hue-sat-value vectors
    M = hsv_to_rgb(hsv)             # convert to RGB
    
    return M

    
def abundancemapToRGB( z ):
    # (JWW 20221208 jesse.wilson@colostate.edu)
    # inspired by Du, et al., "Color Display for Hyperspectral Imagery",
    #             IEEE Trans Geosci and Remote Sens,. v46 no6 (2008).
    
    # generate a color palatte M, one for each endmember,
    # where colors are spaced evely around the color wheel
    ny, nx, nch = z.shape
    M = orthogonalColorPalette(nch)
    
    # the RGB images is simply the tensor dot product between the 
    # abundance map and the color palette,
    # summing over the abundance map's channel axis and 
    # over the color palette's index axis
    rgb = np.tensordot(z,M,(2,0))
    
    return rgb
    
def plotEndmembers( t, w, ax=None ):
    nt, nch = w.shape
    M = orthogonalColorPalette(nch)
    
    if ax==None:
        ax=plt.axes()
    ax.set_facecolor('black')

    for i in range(nch):
        plt.plot(t,w[:,i],color=M[i,:])
    

# Load dataset (CSU mito pump-probe)

In [ ]:
# loading dataset
filenames = os.listdir(data_dir)
img_stack = []
for files in filenames:
    img = io.imread(data_dir + files)
    img_stack.append(img)

# convert images to numpy arrays
img_stack = np.asarray(img_stack)
img_stack = img_stack.astype('float32')

# get image height and width metrics
nimg, nx, ny, nch = img_stack.shape
print('Input dataset specs:')
print('n. images = {}, img width = {}, img height = {}, img channels = {}'.format(nimg, nx, ny, nch))

# process the input image stack
img_stack = torch.tensor(img_stack)
img_stack = img_stack.to(device)


# make sure we have an even number of pixels in width and height
# so that downsampling/upsampling will work correctly
if nx % 2:
    img_stack = img_stack[:,0:(nx-1),:,:]
if ny % 2:
    img_stack = img_stack[:,:,0:(ny-1),:]

In [ ]:
def load_batch_data(img_stack, batch_size, patch_size):
    batch_stack = []
    for i in range(0,batch_size):
        transform = transforms.RandomCrop((patch_size, patch_size))
        img_stack_p = torch.permute(img_stack, (0, 3, 1, 2))
        image_crop = transform(img_stack_p[random.randint(0,22),:,:,:])
#         print(image_crop.size)
        image_crop = torch.permute(image_crop, (1,2,0))
        batch_stack.append(image_crop)
    return batch_stack

def sort_tuple(data):
    data.sort(key = lambda x : x[0])
    return data

In [ ]:
noise = img_stack[:,:,:,0].squeeze().cpu().numpy()
np.mean(noise)
hist,edges = np.histogram(noise, 512)
plt.plot(edges[1:],hist)
plt.xlim([-.1,.1])
plt.show()
print('noise std=  ',np.std(noise))
print('noise mean abs = ', np.mean(np.abs(noise)))

# Load dataset (Duke pigmented lesions)
These are stored as Mathematica .erg format. This code takes David Grass's sample code and wraps it as a pytorch Dataset, following the pytorch tutorial https://pytorch.org/tutorials/beginner/basics/data_tutorial.html.

In [ ]:
from torch.utils.data import Dataset

class DukeMathematicaDataset(Dataset):
    def __init__(self, img_dir):
        self.img_dir = img_dir
        
        dirlisting = os.listdir(img_dir)
        self.file_names = []
        for filename in dirlisting:
            if filename.endswith('.erg'):
                self.file_names.append(filename)
        
        self.t = None
        
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self,idx):
        img_path = os.path.join(self.img_dir,self.file_names[idx])
        
        # open file as read-binary with no buffering
        f = open(img_path, 'rb')

        # import first three int16 which are the time, x and y dimension
        # convert dim to int64, because int16 is not big enougth for
        # multiplications
        dim = np.fromfile(f, dtype=np.int16, count=3)
        dim = dim.astype(np.int64)

        # import the time axis
        self.t = np.fromfile(f, dtype=np.float64, count=dim[0])

        # import stack, loop over time dimension dim[0] and reshape to image
        # dimensions
        images = np.zeros((dim[0],dim[1],dim[2]),dtype=np.float32)
        for i in range(dim[0]):
            temp = np.fromfile(f, dtype=np.float64, count=dim[1]*dim[2])
            images[i,:,:] = temp.reshape((dim[1], dim[2])).astype('float32')

        # close file
        f.close
        
        images = torch.tensor(images)
        images = images.to(device)
        
        return images
    
    def get_t(self):
        # returns time sampling (probe delay) vector for most recently loaded item
        return self.t

In [ ]:
data_dir = '/home/'+username+'/engrprojects/WilsonGroup/hyperspectralCNN/duke_data_20230413/'

dataset = DukeMathematicaDataset(data_dir)
dataset[0].type
dataset.get_t()

In [ ]:
# display an image stack for testing
img = dataset[200].cpu().numpy()
#max_value = np.max(np.abs(img))
max_value = 4
for i in range(len(img)):
    plt.imshow(img[i])
    plt.clim([-max_value, max_value])
    plt.colorbar()
    plt.show()
    clear_output(wait=True)
    

# Synthetic test cases

In [ ]:
# generate synthetic endmembers
nstacks = 100
t=np.array([-2,0,0.2,0.4,1,2,4,6,14])[None,:]
tau=np.array([1,2,3,4,5,6])[:,None]
nch = 9
w = np.exp(-t/tau)*(t>=0)

# generate synthetic abundance maps
ny = 256
nx = 256
blur = 15 # amount of Gaussian blur to apply

nRow = 3
nCol = 3
margin = 8
boxWidth = nx//nCol
boxHeight = ny//nRow
img_stack = []

for k in range(nstacks):

    z = np.zeros((256,256,len(tau)))
    for j in range(len(tau)):
        for i in range(9):
            # for each square
            x = (i % 3)*(nx//nCol)
            y = (i//3)*(ny//nRow)

            if i == j:
                val = 1
            elif i >= len(tau):
                val = np.random.rand() / len(tau)
            else:
                val = 0;

            z[(y+margin//2):(y+boxHeight-margin//2),(x+margin//2):(x+boxWidth-margin//2),j] = val
            
    z = transforms.functional.gaussian_blur(torch.Tensor(z).permute((2,0,1)),blur).permute((1,2,0)).numpy()
    x = np.matmul(z,w)
    x = x + 0.011*np.random.randn(256,256,9)
    img_stack.append(x.astype('float32'))


img_stack = torch.tensor(img_stack)
img_stack = img_stack.to(device)



In [ ]:
plt.figure(figsize=[8,2])
ax = plt.subplot2grid((1,3),(0,0),colspan=2)
plotEndmembers(t.squeeze(),w.T,ax)
plt.subplot2grid((1,3),(0,2))
plt.imshow(abundancemapToRGB(z))
plt.show()


In [ ]:
print(z.shape)
print(w.shape)
plt.figure(figsize=[6,6])
for i in range(t.size):
    plt.subplot(3,3,i+1)
    plt.imshow(img_stack[3][:,:,i].detach().cpu().numpy())
    plt.clim([0,1])
    plt.axis('off')
plt.tight_layout()
plt.show()


# Define functions

In [ ]:
class ResnetBlock(Module):
    def __init__(self, channels, kernel_size, stride, padding):
        super(ResnetBlock, self).__init__()
        self.conv = Sequential(
                        Conv2d(in_channels=channels, 
                          out_channels=channels,
                          kernel_size=kernel_size,
                          stride=stride,
                          padding=padding, bias=False) )
        
    def forward(self, x):
        intermediate = self.conv(x)
        out = x + intermediate
        return out
        

class ScalableSat(Module):
    def __init__(self):
        super(ScalableSat, self).__init__()
 
        self.a = nn.parameter.Parameter(torch.Tensor([1.0,1.0,0.0]))
        self.tanh = Hardtanh()
        #self.tanh = Tanh()
        
    def forward(self,x):
        return self.a[1]*self.tanh(self.a[0]*x)+self.a[2]
    
class LeakyHardsigmoid(Module):
    def __init__(self, negative_slope=0.1):
        super(LeakyHardsigmoid,self).__init__()
        self.ns = negative_slope
        
    def forward(self,x):
        return f.leaky_relu(-f.leaky_relu(-(x-3)/6,negative_slope=self.ns)+1,negative_slope=self.ns)

class OffsetHardsigmoid(Module):
    def __init__(self):
        super(OffsetHardsigmoid,self).__init__()
        
        
    def forward(self,x):
        return f.hardsigmoid(x-3)

class OffsetLeakyHardsigmoid(Module):
    def __init__(self):
        super(OffsetLeakyHardsigmoid,self).__init__()
        
        
    def forward(self,x):
        return f.rrelu(-f.rrelu(-(x-1),lower=0)+1,lower=0)
    
class AbsActivation(Module):
    def __init__(self):
        super(AbsActivation,self).__init__()
        
    def forward(self,x):
        return torch.abs(x)

class NormalizeEndmembers(Module):
    def forward(self,X):
        return X / X.norm(dim=0)
            
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()
        w=32
        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(nch, w, kernel_size=3, stride=1, padding=1,bias=False),
            ReLU(),
#             BatchNorm2d(64),
#             Dropout(0.2),
            Conv2d(w, w, kernel_size=1, stride=1, padding=0,bias=False),
            ReLU(),
#             BatchNorm2d(64),
#             Dropout(0.2),
            Conv2d(w, w, kernel_size=1, stride=1, padding=0,bias=False),
            ReLU(),
#             BatchNorm2d(64),
#             Dropout(0.2),
            Conv2d(w, w, kernel_size=1, stride=1, padding=0,bias=False),
            ReLU(),
#             BatchNorm2d(64),
#             Dropout(0.2),
            Conv2d(w, w, kernel_size=1, stride=1, padding=0,bias=False),
            ReLU(),
#             BatchNorm2d(64),
#             Dropout(0.2),
            
            # Defining additional 2D convolution layer
            Conv2d(w, nch_bottleneck, kernel_size=1, stride=1, padding=0, bias=False),
            
#             nn.Sigmoid(),
            # implementing the v-notch nonlinearity
            nn.Hardtanh(),
            AbsActivation()
            #ReLU()
        )
        
        self.cnn_spectral_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(nch_bottleneck, nch, kernel_size=1, stride=1,bias=False),  
            ScalableSat()
        )
       
        
        self.cnn_layers.apply(initialize_weights_kaiming)
        self.cnn_spectral_layers.apply(initialize_weights_kaiming)
        #self.cnn_spectral_layers.apply(initialize_weights_endmembers)
    
        nn.utils.parametrize.register_parametrization(self.cnn_spectral_layers[0],"weight",NormalizeEndmembers())
        
        
        
    def getSOEMlayer(self):
        return self.cnn_spectral_layers[0]
    
    # Defining the forward pass    
    def forward(self, x):
        z = self.cnn_layers(x)
        y = self.cnn_spectral_layers(z)
        return y
    
    def forward_enc(self, x):
        z = self.cnn_layers(x)

        return z
    
    def forward_dec(self, z):
        y = self.cnn_spectral_layers(z)
        #y = self.cnn_spectral_layers(z + 0.1*torch.randn_like(z))
        return y
    
def initialize_weights_kaiming(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight.data)
        
def initialize_weights_endmembers(m):
    # brownian bridge
    # brownian motion that starts and ends on zero, matching realistic constraints for transient responses
    # equation credit: Diego Krapf
    if isinstance(m, nn.Conv2d):
        nn.init.normal_(m.weight.data,mean=10.)
        #nn.init.uniform_(m.weight.data)
        with torch.no_grad():
            m.weight.data = torch.cumsum(m.weight.data,dim=0)
            m.weight.data[0,:]=0
            m.weight.data = m.weight.data - torch.linspace(0,1,nch)[:,None,None,None]*m.weight.data[-1,:]

        

# define latent space visualization function
def visualize_z(ind):
    data = dataset[ind]
    data = data[None,:]
    data = data.to(device)
    out = model.forward_enc(data)
    out = out[0,:,:,:]
    out = torch.permute(out, (1,2,0))
    img = out.detach().cpu().numpy()
    plt.figure(figsize=(10,10))
    plt.imshow(1.2*img[:,:,0:3])
    plt.show()
    return img

# define latent space visualization function
def generate_z_image(model, ind, indices):
    data = dataset[ind]
    data = data[None,:]
    data = data.to(device)
    out = model.forward_enc(data)
    out = out[0,indices,:,:]
    out = torch.permute(out, (1,2,0))
    img = out.detach().cpu().numpy()
    img = abundancemapToRGB(img)
    return 1.*img

def visualSummary(model, L1_loss, indices):
    plt.figure(figsize=(12,6))
    plt.subplot(2,3,(1,4))
    img1 = generate_z_image(model,0, indices)
    plt.imshow(img1)
    plt.subplot(2,3,(2,5))
    img2 = generate_z_image(model,1, indices)
    plt.imshow(img2)
    ax = plt.subplot(2,3,3)
    w = model.getSOEMlayer().weight
    w = w[:,indices,:,:]
    w = w.detach().cpu().numpy().squeeze()
    #tau=np.array([-2,0,0.2,0.4,1,2,4,6,14])
    tau = dataset.get_t()
    plotEndmembers(tau,w,ax)
    plt.title('recovered endmembers')
    plt.xlabel('probe delay, ps')
    plt.subplot(2,3,6)
    plt.plot(L1_loss)
    plt.title('Train Loss')
    plt.xlabel('Epochs')
    plt.ylabel('L1 Loss')
    plt.tight_layout()
    plt.show()
    return img1, img2
    
# Custom Loss Functions
def meansq(x):
    return torch.mean((x-torch.mean(x))**2)

def corr(x,y):
    return torch.sum(x*y) / (torch.sqrt(torch.sum(x**2))*torch.sqrt(torch.sum(y**2)))

def SADLoss(weights):
    deriv = torch.nn.Parameter(weights[1:,]-weights[:-1,])
    sad = torch.sum(deriv**2)
    return sad

def channelCorrelationLoss(z):
    # inter-channel correlation loss
    zvec = z.squeeze().view(nch_bottleneck,-1)
    zvec = zvec-torch.mean(zvec,1,keepdim=True)
    zcov = torch.matmul(zvec,zvec.transpose(0,1))
    zstds = torch.sqrt(zcov.diag()).expand(1,-1)
    zstdprods = torch.matmul(zstds.transpose(0,1),zstds)
    zcorrabs = zcov / (zstdprods + 1e-6)
    #zcorrabs = torch.max(zcorrabs, torch.tensor(0.5))
    loss_zcorr = (torch.sum((zcorrabs.triu())) - zcorrabs.trace())
    #loss_zcorr = (torch.sum(f.relu(zcorrabs.triu())) - zcorrabs.trace())
    #loss_zcorr += (torch.sum(f.relu(zcov.triu())) - zcov.trace())
    #loss_zcorr = (torch.sum((zcorrabs.triu())) - zcorrabs.trace())
    
    return loss_zcorr

def channelCrossEntropyLoss(z):
    zvec = z.squeeze().view(nch_bottleneck,-1)
    print(zvec.shape)
    loss = f.cross_entropy(zvec[None,0,:],zvec[None,1,:])
    loss += f.cross_entropy(zvec[0,:],zvec[2,:])
    loss += f.cross_entropy(zvec[1,:],zvec[2,:])
    loss = -1.0 * loss
    return loss

    

# model training routine definition
def train_model(model, n_epochs, dataset, lr):
    
    # define loss function
    criterion = nn.MSELoss()
    
    # define optimizer
    #optimizer = torch.optim.Adam([{'params':model.cnn_layers.parameters()},
    #                              {'params':model.cnn_spectral_layers.parameters(),'lr':lr/10.}],
    #                             lr,amsgrad=True)
    optimizer = torch.optim.Adam(model.parameters(),lr)
    #optimizer = torch.optim.AdamW(model.parameters(),lr)
    #optimizer=torch.optim.RMSprop(model.parameters(),lr)
    #optimizer = torch.optim.Adadelta(model.parameters(),lr)
    #optimizer = torch.optim.Adagrad(model.parameters(),lr)
    #optimizer = torch.optim.SGD(model.parameters(),lr)
    
    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',factor=0.9)
    #scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer,lr_lambda= lambda epoch: 0.99)
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=n_epochs,eta_min=0)
    #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr=0.01,total_steps=n_epochs*len(img_stack))
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,50)
    
    
    # initialize model weights and send to train device
    model.to(device) # load the model in the training device
    
    # apply initialization
    
    # define variables
    total_loss = []
    
        
    # iterate over the epochs
    for epoch in tqdm(range(1, n_epochs+1)):
        
        # monitor training loss
        train_loss = 0.0
        loss_recon_avg = 0.
        loss_zcorr_avg = 0.
        
        # keep track of meansq value for each channel
        channelmsq = torch.zeros(nch_bottleneck).to(device)
        
        #Training       
#         current_batch = load_batch_data(img_stack, 150, patch_size)
        
        #img_stack_shuffle  = img_stack[torch.randperm(len(img_stack))]
        for image in dataset:
            #image_x_p = torch.permute(images, (0, 3, 1, 2))
            image_x_p = image[None,:]
            image_x_p = image_x_p.to(device)
            optimizer.zero_grad()   
            z = model.forward_enc(image_x_p)
            outputs = model.forward_dec(z)
            

            # calculate the weighted spectral angular divergence (SAD) loss
    #             in_product = torch.sum(torch.mul(image_x_p, outputs), dim = 1)
    #             self_product = torch.sqrt(torch.sum(torch.mul(image_x_p, image_x_p), dim = 1)) * torch.sqrt(torch.sum(torch.mul(outputs, outputs), dim =1))            
    #             cos_theta = torch.div(in_product, self_product)
    #             theta = torch.acos(cos_theta)
    #             mSAD = torch.mean(theta)

    #             cos = nn.CosineSimilarity(dim=1, eps=1e-8)
    #             cos_loss = torch.mean(torch.acos(cos(image_x_p, outputs)))

            # variance-ordering
            channelmsq += torch.mean(z**2,dim=[2,3]).squeeze()

            # stack sum-squared
            ssq = torch.sum(image_x_p**2, dim = 1)
            ssq = ssq/ torch.sum(ssq)
            ssq = transforms.functional.gaussian_blur(ssq,33)
    #             print('ssq:', ssq)
    
            # calculate the weighted L1 loss
            err_stack = image_x_p - outputs
            chan_weights = 1/torch.std(image_x_p,dim=(2,3))
            chan_weights = chan_weights / torch.sum(chan_weights)
            chan_weights[0] = 0.1
            l1_err_per_chan = torch.mean(torch.abs(err_stack),dim=(2,3))
            spectral_weighted_l1 = torch.mean(chan_weights * l1_err_per_chan)
            
            
            
            # penalize deviation from zero mean error
            loss_zme = torch.mean(torch.mean(err_stack,dim=(2,3))**2)


            loss_recon = criterion(image_x_p, outputs)
            #loss_recon = torch.mean(ssq*(err_stack**2))

            # interchannel correlation loss
            loss_zcorr = channelCorrelationLoss(z)
            #zdown2 = f.avg_pool2d(z,kernel_size=16,stride=1)
            #loss_zcorr = channelCorrelationLoss(zdown2)
            
            #loss_hoyer = torch.mean(torch.sum(torch.abs(z),dim=1)**2 / (torch.sum((z)**2,dim=1)+0.001))
            loss_hoyer = torch.sum(torch.abs(z))**2 / (torch.sum((z)**2)+0.001)
            
            loss_z_l0 = torch.mean(1.0*(z>0.0))
            
            # L1 sparsity on normalized stack
            #ret = z.max(dim=1,keepdim=True)
            #z_normd = z / ret.values
            #z_normd[torch.isnan(z_normd)]=0
            #print(z_normd.shape)
            #print(z_normd)
            z_l1 = torch.mean(torch.abs(z))
    
            # total loss
            #loss = loss_recon + lambda_zcorr * loss_zcorr
            #loss = spectral_weighted_l1 * loss_zcorr
            #loss = loss_recon * loss_zcorr
            #loss = loss_recon + 0.01 * loss_zcorr
            #loss = spectral_weighted_l1
            loss = spectral_weighted_l1+0.001*loss_zcorr
            #loss = spectral_weighted_l1+0.001*loss_zcorr + 1e-8*loss_hoyer
            #loss = spectral_weighted_l1 + 1e-9 * loss_hoyer + 1e-4*loss_zcorr
            #loss = spectral_weighted_l1 + *z_l1
            #loss = loss_recon
            loss.backward()
            optimizer.step()

            #scheduler.step(loss)

            train_loss += loss.item()*image_x_p.size(0)
            
            loss_recon_avg += loss_recon
            loss_zcorr_avg += loss_zcorr
            
            
        train_loss = train_loss/len(dataset)
        total_loss.append(train_loss)
        
        loss_recon_avg = loss_recon_avg / len(dataset)
        loss_zcorr_avg = loss_zcorr_avg / len(dataset)
        
        
        
        #scheduler.step(train_loss)
        
        # rearrange channels so red has max variance, blue the least
        model.indices = channelmsq.argsort(descending=True)
        
        if epoch%1 == 0:
        #if (epoch > 100) and (train_loss == np.min(total_loss)):
        #img1 = 0
        #img2 = 0
            img1, img2 = visualSummary(model,total_loss, model.indices)
            print('recon loss: ', loss_recon_avg.item(), ', zcorr loss: ', loss_zcorr_avg.item())
        
    return total_loss[-1], img1, img2, loss_recon_avg, loss_zcorr_avg


# Train model

In [ ]:
loss_arr = []

# number of times the experiment is repeated
n_runs = 1


nch = len(dataset[0])


n_epochs = 50
learning_rate = 0.01
nch_bottleneck = 2

loss_recon_arr = np.zeros(n_runs)
loss_zcorr_arr = np.zeros(n_runs)


print('Starting training of models on {}'.format(device))
n_count = 0
models = [Net() for i in range(n_runs)]
for i in range(1,n_runs+1):
    #model = Net()
    loss, img1, img2, loss_recon, loss_zcorr = train_model(models[i-1], n_epochs, dataset, learning_rate)
    loss_arr.append((loss, img1, img2))
    loss_recon_arr[i-1] = loss_recon
    loss_zcorr_arr[i-1] = loss_zcorr
    n_count+=1
    print('Runs done: {}/{}'.format(n_count, n_runs))

# Analyze distribution of recovered endmembers

In [ ]:
# generate endmember list
wlist = np.zeros((len(models),9,nch_bottleneck))
for i,model in enumerate(models):
    w = model.getSOEMlayer().weight
    w = w[:,model.indices]
    w = w.detach().cpu().numpy().squeeze()
    wlist[i,:,:] = w
    
wlist = wlist.swapaxes(2,1).reshape([n_runs,9*nch_bottleneck])
wlist[np.isnan(wlist)] = 0

# subtract off means of each column
wmeans = np.mean(wlist,axis=0)
wlistms = wlist - wmeans

# singular value decomposition
u,s,vh = np.linalg.svd(wlistms)

plt.plot(vh[0:3,:].transpose())
plt.xlabel('sample index')
plt.ylabel('signal')
plt.title('eigenvectors')
plt.legend(['1','2','3'])
plt.show()

pltrange= 0.5

# distribution, eigs1 vs 2
plt.figure(figsize=(8,8))
x = np.linspace(-pltrange,pltrange,128)
y = np.linspace(-pltrange,pltrange,128)
#z = np.linspace(-pltrange,pltrange,128)
X,Y = np.meshgrid(x,y)
Z = np.zeros_like(X)
xylist = np.vstack([X.ravel(),Y.ravel(),Z.ravel()])
kernel = stats.gaussian_kde(u[0:3,:])
D = np.reshape(kernel(xylist).T,X.shape)

plt.imshow(D,origin='lower',extent=[x[0],x[-1],y[0],y[-1]])                 

#plt.imshow(H.T,interpolation='nearest',origin='lower',extent=[xedges[0],xedges[-1],yedges[0],yedges[-1]])
plt.scatter(u[0,:],u[1,:],0.5,marker='o',color='white',alpha=0.5)
plt.xlim((-pltrange,pltrange))
plt.ylim((-pltrange,pltrange))
plt.grid()
plt.xlabel('eig 1')
plt.ylabel('eig 2')
plt.title('distribution of all models')
plt.show()


# relative probability of each model
kdeint = kernel.integrate_box([-0.5,-0.5,-0.5],[0.5,0.5,0.5])
print(kdeint)
relprob = kernel.pdf(u[0:3,:])
print(u[0:3,:].shape)
relprob = relprob / np.sum(relprob)


In [ ]:
counts,edges=np.histogram(loss_recon_arr,128)
loss_recon_w = np.sum(relprob*loss_recon_arr)
print('probability-weighted recon loss: ', loss_recon_w)
loss_recon_ml = edges[np.argmax(counts)]
print('most likely recon loss: ', loss_recon_ml)
#aic = 9*np.log(loss_recon_ml)+2*nch_bottleneck
aic = 9*np.log(loss_recon_ml)+9*nch_bottleneck+3
print('AIC: ', aic)

plt.plot(edges[1:],counts)
plt.vlines(loss_recon_w,0,max(counts))
plt.xlabel('reconstruction loss')
plt.ylabel('model count')
plt.title('reconstruction loss histogram')
plt.plot()

loss_recon_w = np.sum(relprob*loss_recon_arr)


In [ ]:
np.max(relprob)

In [ ]:
counts,edges=np.histogram(relprob,128)
plt.plot(edges[1:],counts)
plt.show()
print(np.percentile(relprob,90))
print(np.max(relprob))
print(np.argwhere(relprob > np.percentile(relprob,90)))

# Estimate endmembers and abundances based on relative probability weighting of models

In [ ]:
def generate_z_image_ensembled(ind, models):
    nb,ny,nx,nch = img_stack.shape
    img = np.zeros((len(models),ny,nx,nch_bottleneck))
    for i, model in enumerate(models):
        data = img_stack[ind,:,:,:]
        data = data[None,:]
        data = data.to(device)
        out = model.forward_enc(torch.permute(data, (0, 3, 1, 2)))
        out = out[0,model.indices,:,:]
        out = torch.permute(out, (1,2,0))
        img[i] = relprob[i]*out.detach().cpu().numpy()
        #img[i] = out.detach().cpu().numpy()
        
    #img = img/len(models)
    #img = np.mean(img,axis=0).squeeze()
    img[np.isnan(img)]=0
    img = np.sum(img,axis=0).squeeze()
    #nbin = 50
    #ret = mode((nbin*img).astype(np.int),axis=0)
    #img = ret.mode.astype(np.float)/nbin

    img_recolor = abundancemapToRGB( img )
    return img_recolor

def generate_w_ensembled(models):
    wlist = np.zeros((len(models),9,nch_bottleneck))
    for i,model in enumerate(models):
        w = model.getSOEMlayer().weight
        w = w[:,model.indices]
        w = w.detach().cpu().numpy().squeeze()
        w[np.isnan(w)]=0
        wlist[i,:,:] = relprob[i] * w
        #
        
    return np.sum(wlist,axis=0)
    #return wlist
        
        
tau=np.array([-2,0,0.2,0.4,1,2,4,6,14])
w = generate_w_ensembled(models)
print(w.shape)
alph = 20/len(models)
#alph=0.03
ax=plt.axes()
plotEndmembers(tau,w,ax)
ax.set_xlim([-2,14])
ax.set_ylim([-0.05,0.6])
plt.show()
                     

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_ensembled(0,models))
plt.show()

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_ensembled(1,models))
plt.show()

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_ensembled(2,models))
plt.show()

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_ensembled(5,models))
plt.show()

# Endmembers and abundances based on most likely model

In [ ]:

def generate_z_image_onemodel(ind, model):
    nb,ny,nx,nch = img_stack.shape
    data = img_stack[ind,:,:,:]
    data = data[None,:]
    data = data.to(device)
    out = model.forward_enc(torch.permute(data, (0, 3, 1, 2)))
    out = out[0,model.indices[0:nch_bottleneck],:,:]
    out = torch.permute(out, (1,2,0))
    img = out.detach().cpu().numpy()
    
    img_recolor = abundancemapToRGB( img )
    return img_recolor

def generate_w_onemodel(model):
    w = model.getSOEMlayer().weight
    w = w[:,model.indices]
    w = w.detach().cpu().numpy().squeeze()
    return w

ind_ml = np.argmax(relprob)
print('most likely model: ', ind_ml)

#aic = 9*np.log(loss_recon_arr[ind_ml])+2*nch_bottleneck
aic = 9*np.log(loss_recon_arr[ind_ml])+2*(9*nch_bottleneck)
print('AIC: ', aic)

tau=np.array([-2,0,0.2,0.4,1,2,4,6,14])
w = generate_w_onemodel(models[ind_ml])

ax=plt.axes()
plotEndmembers(tau,w,ax)
ax.set_xlim([-2,14])
ax.set_ylim([-0.05,0.6])
plt.show()
                     

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_onemodel(0,models[ind_ml]))
plt.show()

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_onemodel(1,models[ind_ml]))
plt.show()

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_onemodel(2,models[ind_ml]))
plt.show()

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_onemodel(5,models[ind_ml]))
plt.show()

# Average top most likely models

In [ ]:
def generate_z_image_ensembled(ind, models):
    print(len(models))
    nb,ny,nx,nch = img_stack.shape
    img = np.zeros((len(models),ny,nx,nch_bottleneck))
    for i, model in enumerate(models):
        data = img_stack[ind,:,:,:]
        data = data[None,:]
        data = data.to(device)
        out = model.forward_enc(torch.permute(data, (0, 3, 1, 2)))
        out = out[0,model.indices[0:nch_bottleneck],:,:]
        out = torch.permute(out, (1,2,0))
        img[i] =out.detach().cpu().numpy()
        #img[i] = out.detach().cpu().numpy()
        
    #img = img/len(models)
    img = np.mean(img,axis=0).squeeze()
    img[np.isnan(img)]=0
    #mg = np.median(img,axis=0).squeeze()
    #nbin = 50
    #ret = mode((nbin*img).astype(np.int),axis=0)
    #img = ret.mode.astype(np.float)/nbin
    
    ny,nx,nch = img.shape
    img_recolor = np.zeros((ny,nx,3))
    img_recolor[:,:,0] += img[:,:,0]
    img_recolor[:,:,1] += img[:,:,0]
    img_recolor[:,:,1] += img[:,:,1]
    img_recolor[:,:,2] += img[:,:,1]
    img_recolor[:,:,2] += img[:,:,2]
    img_recolor[:,:,0] += img[:,:,2]
    if nch_bottleneck > 3:
        img_recolor[:,:,0] += img[:,:,3]
    #img_recolor[:,:,1] += img[:,:,4]
    return img_recolor

def generate_w_ensembled(models):
    wlist = np.zeros((len(models),9,nch_bottleneck))
    for i,model in enumerate(models):
        w = model.getSOEMlayer().weight
        w = w[:,model.indices]
        w = w.detach().cpu().numpy().squeeze()
        w[np.isnan(w)]=0
        wlist[i,:,:] = w
        #
        
    return np.mean(wlist,axis=0)
    #return wlist
        
minpr = np.percentile(relprob,90)
models_sel = []
for i in range(n_runs):
    if relprob[i] >= minpr:
        models_sel.append(models[i])

print(len(models_sel))
        
tau=np.array([-2,0,0.2,0.4,1,2,4,6,14])
w = generate_w_ensembled(models_sel)
print(w.shape)
alph = 20/len(models)
#alph=0.03
ax=plt.axes()
ax.set_facecolor('black')
plt.plot(tau,w[:,0],color='yellow')
plt.plot(tau,w[:,1],color='cyan')
plt.plot(tau,w[:,2],color='magenta')
if nch_bottleneck > 3:
    plt.plot(tau,w[:,3],color='red')
    #
#for i in range(len(models)):
#    plt.plot(tau,w[i,:,0],color='yellow',alpha=alph)
#    plt.plot(tau,w[i,:,1],color='cyan',alpha=alph)
#    plt.plot(tau,w[i,:,2],color='magenta',alpha=alph)
    #plt.plot(tau,w[i,:,3],color='red',alpha=alph)
    #plt.plot(tau,w[i,:,4],color='green',alpha=2*alph)
ax.set_xlim([-2,14])
ax.set_ylim([-0.05,0.6])
plt.show()
                     

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_ensembled(0,models_sel))
plt.show()

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_ensembled(1,models_sel))
plt.show()

#plt.figure(figsize=[8,10])
#plt.imshow(generate_z_image_ensembled(2,models))
#plt.show()

#plt.figure(figsize=[8,10])
#plt.imshow(generate_z_image_ensembled(5,models))
#plt.show()

In [ ]:
from itertools import permutations

permobj = permutations([0,1,2,3])
nperms = np.math.factorial(nch_bottleneck)
perms = np.zeros((nperms,nch_bottleneck))
for iperm,perm in enumerate(permobj):
    perms[iperm,:] = perm
    
model = models[1]

w = model.getSOEMlayer().weight
wlist = np.zeros((nperms,9,nch_bottleneck))

for iperm,perm in enumerate(perms):
    wlist[iperm,:,:] = w[:,perm].squeeze().cpu().detach().numpy()
    
wlist = wlist.swapaxes(2,1).reshape([nperms,9*nch_bottleneck])
wlist[np.isnan(wlist)] = 0
    
plt.imshow(wlist)
plt.ylabel('permutation number')
plt.xlabel('pump-probe delay index')
plt.show()

In [ ]:
wlist.shape

In [ ]:
wref = models[ind_ml].getSOEMlayer().weight[:,models[ind_ml].indices].cpu().detach().numpy().squeeze()
wref=wref[None,:]
wref = wref.swapaxes(2,1).reshape([1,9*nch_bottleneck])
wref.shape
plt.plot(wref.T)

In [ ]:
ind_matching_perm = np.argmax(np.sum(wlist * wref,1))
print(ind_matching_perm)
perms[ind_matching_perm]

In [ ]:
perms[12]

In [ ]:
wvarsrt = model.getSOEMlayer().weight[:,model.indices].cpu().detach().numpy().squeeze()
wvarsrt=wvarsrt[None,:]
wvarsrt = wvarsrt.swapaxes(2,1).reshape([1,9*nch_bottleneck])

plt.plot(wref.T)
plt.plot(wlist[12,:].T)
plt.plot(wvarsrt.T)
plt.show()

In [ ]:
# reference model (most likely)
wref = models[ind_ml].getSOEMlayer().weight[:,models[ind_ml].indices].cpu().detach().numpy().squeeze()
wref=wref[None,:]
wref = wref.swapaxes(2,1).reshape([1,9*nch_bottleneck])

# permutations set-up
permobj = permutations([0,1,2,3])
nperms = np.math.factorial(nch_bottleneck)
perms = np.zeros((nperms,nch_bottleneck))
for iperm,perm in enumerate(permobj):
    perms[iperm,:] = perm
    
# iterate through all models
for model in models:

    w = model.getSOEMlayer().weight
    wlist = np.zeros((nperms,9,nch_bottleneck))

    # generate permutations of endmembers for this model
    for iperm,perm in enumerate(perms):
        wlist[iperm,:,:] = w[:,perm].squeeze().cpu().detach().numpy()

    wlist = wlist.swapaxes(2,1).reshape([nperms,9*nch_bottleneck])
    wlist[np.isnan(wlist)] = 0
    
    # find permutation that best matches the reference model
    ind_matching_perm = np.argmax(np.sum(wlist * wref,1))
    
    # store best-matching permutation in the model
    model.indices = perms[ind_matching_perm]
    
    

In [ ]:
def generate_z_image_std(ind, models):
    print(len(models))
    nb,ny,nx,nch = img_stack.shape
    img = np.zeros((len(models),ny,nx,nch_bottleneck))
    for i, model in enumerate(models):
        data = img_stack[ind,:,:,:]
        data = data[None,:]
        data = data.to(device)
        out = model.forward_enc(torch.permute(data, (0, 3, 1, 2)))
        out = out[0,model.indices[0:nch_bottleneck],:,:]
        out = torch.permute(out, (1,2,0))
        img[i] =out.detach().cpu().numpy()
        #img[i] = out.detach().cpu().numpy()
        
    #img = img/len(models)
    img = np.std(img,axis=0).squeeze()
    img[np.isnan(img)]=0
    #mg = np.median(img,axis=0).squeeze()
    #nbin = 50
    #ret = mode((nbin*img).astype(np.int),axis=0)
    #img = ret.mode.astype(np.float)/nbin
    
    ny,nx,nch = img.shape
    img_recolor = np.zeros((ny,nx,3))
    img_recolor[:,:,0] += img[:,:,0]
    img_recolor[:,:,1] += img[:,:,0]
    img_recolor[:,:,1] += img[:,:,1]
    img_recolor[:,:,2] += img[:,:,1]
    img_recolor[:,:,2] += img[:,:,2]
    img_recolor[:,:,0] += img[:,:,2]
    if nch_bottleneck > 3:
        img_recolor[:,:,0] += img[:,:,3]
    #img_recolor[:,:,1] += img[:,:,4]
    return img_recolor

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_std(0,models_sel))
plt.show()

plt.figure(figsize=[8,10])
plt.imshow(generate_z_image_std(1,models_sel))
plt.show()


In [ ]:
for x in loss_arr:
    print("Loss: ", x[0])
    plt.subplot(1,2,1)
    plt.imshow(x[1])
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(x[2])
    plt.axis('off')
    plt.show()

# Analyze trained models

In [ ]:
def basic_boxplot(dataList, expNameList, yRange, metric, dataNum, save=False):
    plt.figure(figsize=(18,6))
    plt.boxplot(dataList)
    plt.ylim(yRange)
    plt.grid()
    plt.xticks([i+1 for i in range(dataNum)], [expNameList[i] + ': ' + str(np.round(np.mean(dataList[i]),6)) for i in range(len(dataList))])
    if str(metric) == 'PSNR':
        plt.ylabel(str(metric) + ' dB')
    else:
        plt.ylabel(str(metric))
    plt.title(str(metric) + ' boxplots for different initialization techniques')
    if save:
        plt.savefig('Boxplot' + str(metric), dpi=100, bbox_inches='tight')
    plt.show()

In [ ]:
dataList = [loss_arr_xu, loss_arr_u, loss_arr_normal, loss_arr_constant, loss_arr_xn, loss_arr_ku, loss_arr_kn]
expNameList = ['xavier_uniform', 'uniform', "normal", 'constant', 'xavier normal', 'kaiming uniform', 'kaiming normal']
basic_boxplot(dataList, expNameList, [0, 0.002], 'Total Loss', 7, False)

# Experiment with new features

### Automated hyperparameter search

In [ ]:
loss_arr = []

# number of times the experiment is repeated
n_runs = 1
n_epochs = 200
learning_rate = 1e-2
nch_bottleneck = 3

print('Starting training of models on {}'.format(device))
n_count = 0
patch_arr = [20, 40, 60, 80, 100, 150, 200, 250]
for patch_size in patch_arr:
    print('Patch Size:', patch_size)
    model = Net()
    loss, img1, img2 = train_model(model, n_epochs, img_stack, learning_rate, patch_size)
    loss_arr.append((loss, img1, img2))
    n_count+=1